In [1]:
import numpy as np
import random

# Parameters Config

In [2]:
SUB      = 0
SUB_ADD  = 1
MULTIPLY = 2

GEN_TYPE = SUB_ADD

TRAINING_SIZE = 80000
DIGITS = 3
ANS_DIGITS = {
    SUB: DIGITS + 1,
    SUB_ADD: DIGITS + 1,
    MULTIPLY: 2 * DIGITS
}.get(GEN_TYPE, DIGITS + 1)
MAXLEN = DIGITS + 1 + DIGITS
chars = {
    SUB: '0123456789- ',
    SUB_ADD: '0123456789+- ',
    MULTIPLY: '0123456789* '
}.get(GEN_TYPE, '0123456789+-* ')

# Data Generation

In [3]:
questions = []
expected = []
seen = set()
operator = {
    SUB: ['-'],
    SUB_ADD: ['-', '+'],
    MULTIPLY: ['*']
}
ans_switcher = {
    '+': lambda a, b: a + b,
    '-': lambda a, b: a - b,
    '*': lambda a, b: a * b
}
ops = operator.get(GEN_TYPE, [None])
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: random.choice(range(10 ** random.choice(range(1, DIGITS + 1))))
    g = lambda: random.choice(ops)
    a, b, op = f(), f(), g()
    if op == '-':
        a, b = sorted((a, b), reverse=True)
    key = tuple((a, b, op))
    if key in seen:
        continue
    seen.add(key)
#     query = '{}{}{}'.format(a, op, b).ljust(MAXLEN)
    query = str(a).rjust(DIGITS) + op + str(b).rjust(DIGITS)
    ans_funct = ans_switcher.get(op, lambda a, b: float('NAN'))
    ans = str(ans_funct(a, b)).rjust(ANS_DIGITS)
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [4]:
for i in range(10):
    print(questions[i], '=', expected[i])

736-  9 =  727
  9+ 95 =  104
590+ 54 =  644
 79-  5 =   74
 38-  1 =   37
 82+  2 =   84
 27+ 30 =   57
  7-  7 =    0
 63- 50 =   13
579-  7 =  572


# Processing
- Size of training data:   64,000
- Size of validation data: 16,000

In [5]:
class CharacterTable:
    def __init__(self, chars):
        self.chars  = list(chars)
        self.len    = len(chars)
        self.encode = {}
        for i, key in enumerate(self.chars):
            self.encode[key] = np.zeros(self.len, np.float32)
            self.encode[key][i] = 1.
            
    def encoder(self, C):
        result = np.zeros((len(C), self.len))
        for i, c in enumerate(C):
            try:
                result[i] = self.encode[c]
            except:
                pass
        return result
            
    def decoder(self, x):
        x = x.argmax(axis=-1)
        return ''.join(self.chars[i] for i in x)

In [6]:
ct = CharacterTable(chars)

x = np.zeros((len(questions), MAXLEN, len(chars)), np.float32)
y = np.zeros((len(expected), ANS_DIGITS, len(chars)), np.float32)
for i, sentence in enumerate(questions):
    x[i] = ct.encoder(sentence)
for i, sentence in enumerate(expected):
    y[i] = ct.encoder(sentence)

train_x = x[:64000]
train_y = y[:64000]

validation_x = x[64000:80000]
validation_y = y[64000:80000]

# Model

![model](img/seq2seq.png)

- Using sequence to sequence model
- Encoder: bi-directional LSTM
- Decoder: LSTM

In [7]:
import keras as K
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Lambda
from keras.layers import Input, LSTM, TimeDistributed, Dropout, RepeatVector, Reshape, Bidirectional, Concatenate, Embedding
from keras.layers.normalization import BatchNormalization
from keras.models import load_model

HIDDEN_SIZE = 256
EMBEDDING_SIZE = 256

model = Sequential()

encoder_inputs = Input(shape=(MAXLEN, len(chars)))
# encoder_inputs = Input(shape=(MAXLEN,))
# embedding = Embedding(len(chars), EMBEDDING_SIZE)(encoder_inputs)
encoder = Bidirectional(LSTM(HIDDEN_SIZE, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
states = [state_h, state_c]

# decoder_inputs = RepeatVector(ANS_DIGITS)(encoder_outputs)
# decoder_outputs = Bidirectional(LSTM(int(HIDDEN_SIZE / 2), return_sequences=True))(decoder_inputs)
# decoder_outputs = TimeDistributed(Dense(len(chars), activation='softmax'))(decoder_outputs)

# Set up the decoder, which will only process one timestep at a time.
decoder_inputs = Reshape((1, HIDDEN_SIZE * 2))
decoder_lstm = LSTM(HIDDEN_SIZE * 2, return_state=True)

all_outputs = []
inputs = decoder_inputs(encoder_outputs)

first_decoder = True
for _ in range(ANS_DIGITS):
    # Run the decoder on one timestep
    outputs, state_h, state_c = decoder_lstm(inputs,
                                             initial_state=states)
    
    # Reinject the outputs as inputs for the next loop iteration
    # as well as update the states
    states = [state_h, state_c]
#     inputs = decoder_inputs(outputs)
    
    # Store the current prediction (we will concatenate all predictions later)
    outputs = Dense(len(chars), activation='softmax')(outputs)
    all_outputs.append(outputs)
    

# Concatenate all predictions
decoder_outputs = Concatenate()(all_outputs)
decoder_outputs = Reshape((ANS_DIGITS, len(chars)))(decoder_outputs)
decoder_outputs = Lambda(lambda x: x[:,::-1])(decoder_outputs)

# Define and compile model as previously
model = Model(encoder_inputs, decoder_outputs)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

batch_size = int(len(train_x) / 128 / 100) * 100

if batch_size == 0:
    batch_size = 100

    
model.fit(train_x, train_y, 
          batch_size=batch_size, epochs=100, 
          verbose=1, validation_data=[validation_x, validation_y])

# model.fit(np.argmax(train_x, axis=2), train_y, 
#           batch_size=batch_size, epochs=100, 
#           verbose=1, validation_data=[np.argmax(validation_x, axis=2), validation_y])

/home/ddnn_user02/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7, 13)        0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 512), (None, 552960      input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 512)       0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 512)          0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]            
__________

64000/64000 [==============================] - 6s 101us/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0027 - val_acc: 0.9998
Epoch 27/100
64000/64000 [==============================] - 6s 101us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0025 - val_acc: 0.9998
Epoch 28/100
64000/64000 [==============================] - 6s 101us/step - loss: 9.3872e-04 - acc: 1.0000 - val_loss: 0.0023 - val_acc: 0.9997
Epoch 29/100
64000/64000 [==============================] - 7s 102us/step - loss: 8.3903e-04 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 0.9998
Epoch 30/100
64000/64000 [==============================] - 7s 103us/step - loss: 7.5261e-04 - acc: 1.0000 - val_loss: 0.0020 - val_acc: 0.9998
Epoch 31/100
64000/64000 [==============================] - 6s 101us/step - loss: 6.7740e-04 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 0.9997
Epoch 32/100
64000/64000 [==============================] - 7s 103us/step - loss: 6.1113e-04 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 0.9998
Epoch 33/100


Epoch 82/100
64000/64000 [==============================] - 7s 104us/step - loss: 1.1146e-05 - acc: 1.0000 - val_loss: 3.4140e-04 - val_acc: 0.9999
Epoch 83/100
64000/64000 [==============================] - 7s 103us/step - loss: 1.0378e-05 - acc: 1.0000 - val_loss: 3.6021e-04 - val_acc: 0.9998
Epoch 84/100
64000/64000 [==============================] - 6s 100us/step - loss: 9.6933e-06 - acc: 1.0000 - val_loss: 3.2435e-04 - val_acc: 0.9998
Epoch 85/100
64000/64000 [==============================] - 6s 101us/step - loss: 9.0426e-06 - acc: 1.0000 - val_loss: 3.4797e-04 - val_acc: 0.9999
Epoch 86/100
64000/64000 [==============================] - 7s 102us/step - loss: 8.4410e-06 - acc: 1.0000 - val_loss: 3.7091e-04 - val_acc: 0.9998
Epoch 87/100
64000/64000 [==============================] - 6s 101us/step - loss: 7.8817e-06 - acc: 1.0000 - val_loss: 3.7478e-04 - val_acc: 0.9999
Epoch 88/100
64000/64000 [==============================] - 7s 102us/step - loss: 7.3644e-06 - acc: 1.0000 - val

# Error check

In [8]:
print('Error check')
pred = model.predict(x)
err = []
for i in range(len(x)):
    if ct.decoder(pred[i]) != ct.decoder(y[i]):
        err.append(i)
print(len(err), '/', len(x))
print()
print('Prediction'.ljust(MAXLEN + ANS_DIGITS + 3),'Ans')
print('-' * (MAXLEN + ANS_DIGITS * 2 + 4))
for i in err:
    print(ct.decoder(x[i]), '=', ct.decoder(pred[i]), ct.decoder(y[i]))


Error check
7 / 80000

Prediction     Ans
-------------------
341-236 =   05  105
959+680 = 1539 1639
936+155 = 1191 1091
912+484 = 1496 1396
915-807 =   08  108
913+995 = 1808 1908
603-502 =   01  101


# Test

In [9]:
q = '100-5'

q_padding = q.ljust(MAXLEN)[:MAXLEN]
test_x = ct.encoder(q_padding)
pred_y = model.predict(test_x.reshape(-1, MAXLEN, len(chars)))
print(q, '=', ct.decoder(pred_y[0]))


100-5 =   98
